### Evaluation
이 예제에서는 Evaluation을 해보겠습니다. `rouge`같은 벤치마크가 아닌 LLM을 이용한 평가 방법입니다.  
물론 벤치마크 평가가 사장된 것은 아닙니다만, 현재는 LLM을 통한 평가 방법을 일반적으로 많이 사용하고 있습니다.  

이 예제는 [https://arxiv.org/pdf/2306.05685](https://arxiv.org/pdf/2306.05685)와 [evals](https://github.com/openai/evals)바탕으로 작성되었습니다.  
프롬프트 또한 논문을 기반으로 작성되었습니다.  

평가는 아래의 순서로 진행됩니다.  
1. 튜닝전 모델과 튜닝 후 모델을 불러옵니다.
2. 작성해놓은 평가 프롬프트를 input으로 하여 각각의 모델로 output을 생성합니다.
3. ChatGPT-4o 각 모델들의 output을 전달하여 평가하도록 요청합니다.

In [ ]:
!pip install --quiet\
accelerate\
flash-attn\
transformers\
openai

In [ ]:
import os
import sys
import json
import copy
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from google.colab import userdata, drive
# utils, prompts 커스텀 모듈 사용을 위해 구글 드라이브에 마운트합니다.
drive.mount('/content/drive')

# git clone 받은 위치로 지정합니다.
path = "/content/drive/MyDrive/instruction-tuning-with-rag-example"

sys.path.append(path)
import utils
import prompts

### 1. 튜닝전 모델과 튜닝 후 모델을 불러옵니다

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
base_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    token=userdata.get("HF_TOKEN"),
    attn_implementation="flash_attention_2"
)

finetuned_model = AutoModelForCausalLM.from_pretrained(
    os.path.join(path, "gemma-2b-it-example-v1"),
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)

### 2. 작성해놓은 평가 프롬프트를 input으로 하여 각각의 모델로 output을 생성합니다.

In [ ]:
# 평가 데이터 불러오기
with open(os.path.join(path, "data/eval_dataset.txt"), "r") as f:
    eval_inputs = f.readlines()

eval_inputs = [inputs.strip("\n") for inputs in eval_inputs]
eval_inputs

In [ ]:
# model.generate
inputs = eval_inputs
inputs_with_tag = [instruction + "<ANSWER>" for instruction in inputs]

input_text = tokenizer(inputs_with_tag, return_tensors="pt", padding=True).to(model.device)

base_outputs = model.generate(**input_text, max_new_tokens=512, repetition_penalty = 1.5)
base_decoded = tokenizer.batch_decode(base_outputs, skip_special_tokens=True)
base_decoded = [decoded.split("<ANSWER>")[1] for decoded in base_decoded]

finetuned_outputs = finetuned_model.generate(**input_text, max_new_tokens=512, repetition_penalty = 1.5)
finetuned_decoded = tokenizer.batch_decode(finetuned_outputs, skip_special_tokens=True)
finetuned_decoded = [decoded.split("<ANSWER>")[1] for decoded in finetuned_decoded]

eval_dataset = dict(inputs=inputs, completion1=base_decoded, completion2=finetuned_decoded)
utils.jsave(data=eval_dataset, save_path=os.path.join(path, "data/eval_dataset.json"), mode="w", indent=4)

In [ ]:
eval_dataset = utils.jload("./data/eval_dataset.json")

In [ ]:
for i1, c1, i2, c2 in zip(eval_dataset['input1'], eval_dataset['completion1'], eval_dataset['input2'], eval_dataset['completion2']):
    utils.jsave(dict(input1=i1, completion1=c1, input2=i2, completion2=c2), "./data/eval_dataset.jsonl", "a")
    with open("./data/eval_dataset.jsonl", "a") as f:
        f.write("\n")

In [ ]:
eval_dataset = utils.jload("./data/eval_dataset.jsonl")

In [ ]:
scores = []
for evals in eval_dataset:
    data = json.loads(evals)
    res = utils.get_completion(prompts.EVAL_BATTLE_PROMPT.format(question=data['input1'], answer_a=data['completion1'], answer_b=data['completion2']))
    scores.append(res)

In [ ]:
scores = []
for evals in eval_dataset:
    data = json.loads(evals)
    res = utils.get_completion(prompts.EVAL_SCORE_PROMPT.format(question=data['input1'], answer_a=data['completion1'], answer_b=data['completion2']))
    scores.append(res)